<a href="https://colab.research.google.com/github/aiAintEasy-HolyCalamity/SupervisedML_HousingPricePrediction/blob/main/PredictiveAnalyticsForStockPrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install yfinance pandas numpy matplotlib scikit-learn tensorflow newsapi-python vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.3 MB/s eta 0:00:00


In [2]:
#Import Libraries

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from newsapi import NewsApiClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [3]:
# Fetch stock data (Example: Apple stock)
ticker = 'AAPL'
stock_data = yf.download(ticker, start='2010-01-01', end='2025-01-01')

# Display the first few rows of the data
stock_data.head()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.440329,6.455075,6.391277,6.422875,493729600
2010-01-05,6.451463,6.487876,6.417457,6.458084,601904800
2010-01-06,6.348848,6.477047,6.342227,6.451467,552160000
2010-01-07,6.337109,6.379843,6.291066,6.372319,477131200
2010-01-08,6.379241,6.379843,6.291368,6.328683,447610800


In [5]:
# Simple Moving Average (SMA)
stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()
stock_data['SMA_200'] = stock_data['Close'].rolling(window=200).mean()

# Exponential Moving Average (EMA)
stock_data['EMA_50'] = stock_data['Close'].ewm(span=50, adjust=False).mean()
stock_data['EMA_200'] = stock_data['Close'].ewm(span=200, adjust=False).mean()  # Added EMA_200 calculation

# Moving Average Convergence Divergence (MACD)
stock_data['MACD'] = stock_data['EMA_50'] - stock_data['EMA_200']
stock_data['Signal_Line'] = stock_data['MACD'].ewm(span=9, adjust=False).mean()

# Relative Strength Index (RSI)
delta = stock_data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
stock_data['RSI'] = 100 - (100 / (1 + rs))

# Display the data with technical indicators
stock_data.tail()


Price,Close,High,Low,Open,Volume,SMA_50,SMA_200,EMA_50,EMA_200,MACD,Signal_Line,RSI
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,,,,,,,
Date,,,,,,,,,,,,
2024-12-24,257.916443,257.926411,255.009620,255.209412,23234700,NaN,NaN,NaN,216.974667,NaN,NaN,76.180657
2024-12-26,258.735504,259.814335,257.347047,257.906429,27237100,NaN,NaN,NaN,217.390197,NaN,NaN,76.812079
2024-12-27,255.309296,258.415896,252.782075,257.546826,42355300,NaN,NaN,NaN,217.767502,NaN,NaN,69.300685
2024-12-30,251.923019,253.221595,250.474615,251.952985,35557500,NaN,NaN,NaN,218.107358,NaN,NaN,58.382055
2024-12-31,250.144974,253.001833,249.156056,252.162760,39480700,NaN,NaN,NaN,218.426140,NaN,NaN,53.982582


#Step 5

In [17]:
from newsapi import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

import nltk
nltk.download('vader_lexicon')

# Now you can use SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize the NewsAPI client (replace with your actual API key)
news_api = NewsApiClient(api_key='d53b3306d3f44b958f9dcf758a4f63a3')

# Set ticker symbol (example: 'AAPL' for Apple)
ticker = 'AAPL'

# Fetch articles related to the stock ticker
response = news_api.get_everything(q=ticker, language='en', sort_by='publishedAt')

# Sentiment analysis with VADER
analyzer = SentimentIntensityAnalyzer()

# Extract and analyze sentiment for each article
news_sentiment = []

# Check if 'articles' exist in the response
if 'articles' in response:
    articles = response['articles']  # Get the articles list
    for article in articles:
        # Get title and description (default to empty string if None)
        title = article.get('title', '')
        description = article.get('description', '')

        # Ensure both are strings and concatenate them
        text = (title if title else '') + ' ' + (description if description else '')

        # Perform sentiment analysis if there is text to analyze
        if text.strip():  # Only proceed if there's actual content to analyze
            sentiment_score = analyzer.polarity_scores(text)['compound']
            news_sentiment.append(sentiment_score)

    # Calculate the average sentiment score for the news
    if news_sentiment:  # Ensure there's at least one sentiment score to average
        average_sentiment = np.mean(news_sentiment)
        print(f"Average sentiment score for {ticker}: {average_sentiment}")
    else:
        print("No valid articles to analyze.")
else:
    print("Error: No articles found in the response.")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Average sentiment score for AAPL: 0.517011


#Step 6

In [19]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

# Download the VADER lexicon (if not already downloaded)
nltk.download('vader_lexicon')

# Initialize the NewsAPI client (replace with your actual API key)
from newsapi import NewsApiClient
news_api = NewsApiClient(api_key='d53b3306d3f44b958f9dcf758a4f63a3')

# Set ticker symbol (example: 'AAPL' for Apple)
ticker = 'AAPL'

# Fetch articles related to the stock ticker
response = news_api.get_everything(q=ticker, language='en', sort_by='publishedAt')

# Sentiment analysis with VADER
analyzer = SentimentIntensityAnalyzer()

# Extract and analyze sentiment for each article
news_sentiment = []

# Check if 'articles' exists in the response
if 'articles' in response:
    articles = response['articles']  # Get the articles list
    for article in articles:
        # Get title and description (default to empty string if None)
        title = article.get('title', '')
        description = article.get('description', '')

        # Ensure both are strings and concatenate them
        text = (title if title else '') + ' ' + (description if description else '')

        # Perform sentiment analysis if there is text to analyze
        if text.strip():  # Only proceed if there's actual content to analyze
            sentiment_score = analyzer.polarity_scores(text)['compound']
            news_sentiment.append(sentiment_score)

    # Calculate the average sentiment score for the news
    if news_sentiment:  # Ensure there's at least one sentiment score to average
        average_sentiment = np.mean(news_sentiment)
        print(f"Average sentiment score for {ticker}: {average_sentiment}")
    else:
        print("No valid articles to analyze.")
else:
    print("Error: No articles found in the response.")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Average sentiment score for AAPL: 0.517011


#Step 7

In [22]:
import yfinance as yf
import pandas as pd
import numpy as np
import nltk
from sklearn.preprocessing import MinMaxScaler

# Ensure the VADER lexicon is downloaded (required for sentiment analysis)
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Load stock data (example: AAPL)
ticker = 'AAPL'
df = yf.download(ticker, start="2020-01-01", end="2025-01-01")

# Step 7: Feature Engineering

# Add technical indicators (Moving Averages, MACD, etc.)
def add_technical_indicators(df):
    # Moving Averages
    df['SMA_50'] = df['Close'].rolling(window=50).mean()  # 50-day SMA
    df['SMA_200'] = df['Close'].rolling(window=200).mean()  # 200-day SMA
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()  # 50-day EMA
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()  # 200-day EMA

    # MACD (Moving Average Convergence Divergence)
    df['MACD'] = df['EMA_50'] - df['EMA_200']  # MACD Line
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()  # Signal Line

    # Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    return df

# Add news sentiment as a feature (assuming `average_sentiment` has been calculated previously)
average_sentiment = 0.1  # Example value for news sentiment; replace with actual value
df['News_Sentiment'] = average_sentiment

# Add technical indicators to the dataframe
df = add_technical_indicators(df)

# Check for any missing values in the dataframe
print(f"Missing values in data: {df.isnull().sum()}")

# Drop rows with missing values (optional: you can use other imputation methods)
df = df.dropna()

# Feature scaling (Optional: Normalize the features)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['Close', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'MACD', 'Signal_Line', 'RSI', 'News_Sentiment']])

# Prepare the final dataset with scaled features
scaled_df = pd.DataFrame(scaled_features, columns=['Close', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'MACD', 'Signal_Line', 'RSI', 'News_Sentiment'])

# Check the final dataframe
print(scaled_df.head())


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[*********************100%***********************]  1 of 1 completed

Missing values in data: Price           Ticker
Close           AAPL        0
High            AAPL        0
Low             AAPL        0
Open            AAPL        0
Volume          AAPL        0
News_Sentiment              0
SMA_50                     49
SMA_200                   199
EMA_50                      0
EMA_200                     0
MACD                        0
Signal_Line                 0
RSI                        13
dtype: int64
      Close    SMA_50   SMA_200    EMA_50   EMA_200      MACD  Signal_Line  \
0  0.076258  0.014160  0.000000  0.000000  0.000000  0.871851     0.896373   
1  0.065465  0.014990  0.001711  0.001675  0.001859  0.871318     0.896721   
2  0.046049  0.015749  0.003333  0.002390  0.003466  0.867792     0.896264   
3  0.055821  0.016495  0.004991  0.003526  0.005175  0.865620     0.895446   
4  0.051733  0.017664  0.006638  0.004430  0.006817  0.862743     0.894192   

        RSI  News_Sentiment  
0  0.644442             0.0  
1  0.573552          

#Step 8

In [34]:
import yfinance as yf
import pandas as pd
import numpy as np
import nltk
from sklearn.preprocessing import MinMaxScaler

# Ensure the VADER lexicon is downloaded (required for sentiment analysis)
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Load stock data (example: AAPL)
ticker = 'AAPL'
df = yf.download(ticker, start="2020-01-01", end="2025-01-01")

# Step 7: Feature Engineering

# Add technical indicators (Moving Averages, MACD, etc.)
def add_technical_indicators(df):
    # Moving Averages
    df['SMA_50'] = df['Close'].rolling(window=50).mean()  # 50-day SMA
    df['SMA_200'] = df['Close'].rolling(window=200).mean()  # 200-day SMA
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()  # 50-day EMA
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()  # 200-day EMA

    # MACD (Moving Average Convergence Divergence)
    df['MACD'] = df['EMA_50'] - df['EMA_200']  # MACD Line
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()  # Signal Line

    # Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    return df

# Add news sentiment as a feature (assuming `average_sentiment` has been calculated previously)
average_sentiment = 0.1  # Example value for news sentiment; replace with actual value
df['News_Sentiment'] = average_sentiment

# Add technical indicators to the dataframe
df = add_technical_indicators(df)

# Check for any missing values in the dataframe
print(f"Missing values in data: {df.isnull().sum()}")

# Drop rows with missing values (optional: you can use other imputation methods)
df = df.dropna()

# Feature scaling (Optional: Normalize the features)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['Close', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'MACD', 'Signal_Line', 'RSI', 'News_Sentiment']])

# Prepare the final dataset with scaled features
scaled_df = pd.DataFrame(scaled_features, columns=['Close', 'SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'MACD', 'Signal_Line', 'RSI', 'News_Sentiment'], index=df.index)  # Use df.index to keep the original index

# Now, create the 'Target' column on the scaled DataFrame
scaled_df['Target'] = scaled_df['Close'].shift(-1)

# Drop rows with NaN values in the 'Target' column in the scaled DataFrame
scaled_df.dropna(subset=['Target'], inplace=True)

# --- Rest of your code using scaled_df ---

# Check the final dataframe
print(scaled_df.head())

# Check DataFrame before creating 'Target'
print("DataFrame columns before creating 'Target':", scaled_df.columns)  # Check existing columns

# Debugging: Check if 'Target' column exists after creation
print("DataFrame columns after creating 'Target':", scaled_df.columns)  # Should include 'Target'

# Check the first few rows to confirm 'Target' was created
print("First few rows of 'Close' and 'Target':")
print(scaled_df[['Close', 'Target']].head())

# Check if there are any NaN values in the 'Target' column
print("Missing values in 'Target' column:", scaled_df['Target'].isnull().sum())


# Now proceed with the rest of your model building steps
# Make sure necessary libraries are imported
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # Import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score  # Import r2_score

X = scaled_df[['SMA_50', 'SMA_200', 'EMA_50', 'EMA_200', 'MACD', 'Signal_Line', 'RSI', 'News_Sentiment']]
y = scaled_df['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()  # Initialize LinearRegression model
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)  # Calculate R-squared

# Print the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[*********************100%***********************]  1 of 1 completed


Missing values in data: Price           Ticker
Close           AAPL        0
High            AAPL        0
Low             AAPL        0
Open            AAPL        0
Volume          AAPL        0
News_Sentiment              0
SMA_50                     49
SMA_200                   199
EMA_50                      0
EMA_200                     0
MACD                        0
Signal_Line                 0
RSI                        13
dtype: int64
               Close    SMA_50   SMA_200    EMA_50   EMA_200      MACD  \
Date                                                                     
2020-10-15  0.076258  0.014160  0.000000  0.000000  0.000000  0.871851   
2020-10-16  0.065465  0.014990  0.001711  0.001675  0.001859  0.871318   
2020-10-19  0.046049  0.015749  0.003333  0.002390  0.003466  0.867792   
2020-10-20  0.055821  0.016495  0.004991  0.003526  0.005175  0.865620   
2020-10-21  0.051733  0.017664  0.006638  0.004430  0.006817  0.862743   

            Signal_Line       R